# IMDb Scraper

Workflow: 5 

Goal: Start with TMDB ids and get all data from IMDB.

Result: The file ```imdb_movie.csv``` is created.

In [1]:
import os
import time
import pandas as PD

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
# can't reload script in a different folder

import sys
sys.path.append('../models')
import imdb_scraper as MB 

## Scrape One Movie

In [4]:
scraper = MB.ImdbScraper()
title = 'Crouching Tiger, Hidden Dragon: Sword of Destiny'
year = '2016'

<selenium.webdriver.firefox.webdriver.WebDriver (session="1d476b86-63a0-0e46-9707-53147b893a72")>


In [5]:
# scraper.open_imdb()
# scraper.search_any(title)

In [6]:
# scraper.find_movie_result(title, year)

In [7]:
# movie_dx = scraper.get_movie_data(title)
# movie_dx 

In [8]:
scraper.close()

## Scrape All Netflix Movies

In [9]:
INDEX_FILE = '../data/moviedb_subsample.csv'
index_df = PD.read_csv(INDEX_FILE)
index_df.head()
index_df.info()

,title,original_title,year,companies,country,language,runtime,crew,cast,poster,genres,collection,synopsis,budget,gross,tmdb_score,tmdb_votes,tmdb_id,imdb_id
0,1,1,2013,"Exclusive Media, Flat-Out Films",United States of America,English,112.0,"Paul Crowder, Mark Monroe, Michael Shevloff","Niki Lauda, Michael Schumacher, Lewis Hamilton",/4uIPXX8DjTsCzUAdtMKHTpojYLq.jpg,Documentary,NaN,Set in the golden era of Grand Prix Racing '1'...,NaN,NaN,7.4,59,217316,tt2518788
1,13 Times Evil,13 Times Evil,2016,NaN,NaN,English,NaN,"Philip Gardiner, O.H. Krill","Rick Wallen, Charles Thompsen, Robert Pitt",/tcdAJMNPAlqtsNPtYK0F0GUgHKG.jpg,Documentary,NaN,History is replete with psychotic killers with...,NaN,NaN,0.0,0,386666,NaN
2,21 Days,21 Days,2014,"Nopal Army, Tortilla Flats Productions",United States of America,English,89.0,Kathleen Behun,"Max Hambleton, Whitney Rose Pynn, Mickey River",/1IXt1E6i6tH8gaCFG6jhlsKM4V5.jpg,"Thriller, Horror",NaN,Three filmmakers embark on a paranormal challe...,NaN,NaN,4.7,10,341957,tt2979366
3,36 Hour Layover,36 Hour Layover,2016,RLJ Entertainment,United States of America,English,88.0,Mark Harris,"Amin Joseph, Monica Calhoun, Brian J. White",/vZCoqQecIEGqfDr4Q3Ppvg3ZXIF.jpg,"Comedy, Romance",NaN,This film is a romantic comedy about a steward...,1000000.0,NaN,10.0,1,441453,tt5201246
4,666: Devilish Charm,666: Devilish Charm,2014,Rapid Heart Pictures,United States of America,English,75.0,"David DeCoteau, Rolfe Kanefsky","Courtney B Turk, Ryan Walker, R.J. Cantu",/cauliUwjBwVEFWGBhlpjnBmWec5.jpg,"Horror, Thriller",NaN,When a cursed devil's charm' bracelet shows up...,1000000.0,NaN,3.8,5,270802,tt3216846


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 19 columns):
title             319 non-null object
original_title    319 non-null object
year              319 non-null int64
companies         214 non-null object
country           245 non-null object
language          319 non-null object
runtime           290 non-null float64
crew              303 non-null object
cast              288 non-null object
poster            311 non-null object
genres            285 non-null object
collection        22 non-null object
synopsis          317 non-null object
budget            57 non-null float64
gross             29 non-null float64
tmdb_score        319 non-null float64
tmdb_votes        319 non-null int64
tmdb_id           319 non-null int64
imdb_id           314 non-null object
dtypes: float64(4), int64(3), object(12)
memory usage: 47.5+ KB


In [10]:
scraper = MB.ImdbScraper()
scraper.open_imdb()
movie_ls = []
t0 = time.time()

for idx, row in index_df.iterrows():
    title = row['title']
    year = row['year']
    imdb_id = row['imdb_id']
    
    try:
#         scraper.search_any(title)
#         found = scraper.find_movie_result(title, year)
#         if not found:
#             print(f'Not found: {title} {year}')
#             continue
        
#         movie_dx = scraper.get_movie_data(title)

        movie_dx = scraper.get_movie_by_id(imdb_id)
        
        # if not by id, then try title-year
    
        movie_ls.append(movie_dx)
    except:
        print(f'Error: {title} {year}')

scraper.close()
t1 = time.time()
print(f'movies: {len(movie_ls)}')
print(f'time: {(t1-t0)/60:.2f} mins')

<selenium.webdriver.firefox.webdriver.WebDriver (session="7fb666a4-e79b-e043-ac01-d23e3dbc77f0")>
Error: 13 Times Evil 2016
Error: Kaaliyan 2017
Error: Metallica: Kill 'Em All to St. Anger - The Ultimate Critical Review 2006
Error: There Is a New World Somewhere 2015
Error: The Riverside Murder 1935
Error: The Scarlet and the Black 1983
Error: The Sex Trip 2017
Error: The Sleeping Tiger 1954
Error: The Star Next Door 2017
Error: The Swan Princess 1994
Error: The Thing Below 2004
Error: The Tree of Blood 2018
Error: The Untold Story 1993
Error: The Wasp Woman 1959
Error: The Wiggles - Taking Off! 2013
Error: The Worst Year of My Life 2015
Error: Thirst 2016
Error: Thug Love 2009
Error: Time Trap 2017
Error: To Love The Soul Of A Woman 2017
Error: Top Ten Monks 2010
Error: Traffic 2016
Error: Tree Man 2016
Error: True Friends 1998
Error: Turning Point 2011
Error: Udanchhoo 2018
Error: Under Jakob's Ladder 2011
Error: Unmade Beds 2009
Error: Uuf Kya Jadoo Mohobbat Hai 2004
Error: Vegucate

In [11]:
movie_df = PD.DataFrame(movie_ls)
movie_df.head()
movie_df.info()

,imdb_id,title_imdb,title_original,year,rating,company,country,language,runtime,director,writer,actors,genres,synopsis,budget,gross_us,gross_worldwide,score,votes
0,tt2518788,1,None,2013,Not Rated,Diamond Docs,USA,English,112.0,Paul Crowder,Mark Monroe,"Michael Fassbender, Niki Lauda, Lewis Hamilton","Documentary, History, Sport",Set in the golden era of Grand Prix Racing '1'...,None,None,None,8.0,3534
1,tt2979366,21 Days,None,2014,TV-MA,Tortilla Flats Productions,USA,English,89.0,Kathleen Behun,Kathleen Behun,"Max Hambleton, Whitney Rose Pynn, Mickey River","Horror, Thriller",Three filmmakers embark on a paranormal challe...,None,None,None,4.5,493
2,tt5201246,No Regrets,36 Hour Layover,2016,Not Rated,None,USA,English,88.0,Mark Harris,Mark Harris,"Loretta Devine, Sharon Leal, Elise Neal",Comedy,After 12 years with her college sweetheart (an...,1000000,None,None,5.8,74
3,tt3216846,Devilish Charm,None,2014,Not Rated,Rapid Heart Pictures,USA,English,78.0,David DeCoteau,Rolfe Kanefsky,"Courtney Blythe Turk, Ryan Walker, R.J. Cantu",Thriller,When a cursed devil's charm' bracelet shows up...,1000000,None,None,3.4,110
4,tt0306434,Aankhen,None,2002,TV-14,Gaurang Doshi Productions,India,Hindi,165.0,Vipul Amrutlal Shah,Shobhna Desai,"Amitabh Bachchan, Akshay Kumar, Sushmita Sen","Comedy, Crime, Drama",After a temperamental man is unceremoniously s...,None,None,None,7.4,11512


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275 entries, 0 to 274
Data columns (total 19 columns):
imdb_id            275 non-null object
title_imdb         275 non-null object
title_original     40 non-null object
year               275 non-null object
rating             199 non-null object
company            248 non-null object
country            272 non-null object
language           271 non-null object
runtime            263 non-null float64
director           275 non-null object
writer             246 non-null object
actors             257 non-null object
genres             275 non-null object
synopsis           275 non-null object
budget             85 non-null object
gross_us           48 non-null object
gross_worldwide    87 non-null object
score              273 non-null object
votes              273 non-null object
dtypes: float64(1), object(18)
memory usage: 40.9+ KB


In [12]:
save_path = r'../data/imdb_subsample.csv'
movie_df.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'

'0.1 mb'